# Downloads and Installations

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
!pip install spacy[transformers,cuda111]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 460.3 MB 21 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [ ]:
#!pip install spacy-transformers

In [ ]:
#Check that the transformers package is installed
import spacy
nlp = spacy.load('en_core_web_trf')

In [ ]:
!wget https://nlp.stanford.edu/software/stanford-ner-4.2.0.zip

--2022-06-30 12:31:26--  https://nlp.stanford.edu/software/stanford-ner-4.2.0.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://downloads.cs.stanford.edu/nlp/software/stanford-ner-4.2.0.zip [following]
--2022-06-30 12:31:27--  https://downloads.cs.stanford.edu/nlp/software/stanford-ner-4.2.0.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180437064 (172M) [application/zip]
Saving to: ‘stanford-ner-4.2.0.zip’

stanford-ner-4.2.0. 100%[===================>] 172.08M  5.08MB/s    in 31s     

2022-06-30 12:31:58 (5.54 MB/s) - ‘stanford-ner-4.2.0.zip’ saved [180437064/180437064]



In [ ]:
!unzip /content/stanford-ner-4.2.0.zip

Archive:  /content/stanford-ner-4.2.0.zip
   creating: stanford-ner-2020-11-17/
   creating: stanford-ner-2020-11-17/lib/
  inflating: stanford-ner-2020-11-17/lib/jollyday-0.4.9.jar  
  inflating: stanford-ner-2020-11-17/lib/stanford-ner-resources.jar  
  inflating: stanford-ner-2020-11-17/lib/joda-time.jar  
  inflating: stanford-ner-2020-11-17/stanford-ner-4.2.0.jar  
  inflating: stanford-ner-2020-11-17/NERDemo.java  
  inflating: stanford-ner-2020-11-17/LICENSE.txt  
  inflating: stanford-ner-2020-11-17/sample-conll-file.txt  
  inflating: stanford-ner-2020-11-17/stanford-ner-4.2.0-javadoc.jar  
  inflating: stanford-ner-2020-11-17/stanford-ner-4.2.0-sources.jar  
  inflating: stanford-ner-2020-11-17/stanford-ner.jar  
  inflating: stanford-ner-2020-11-17/sample.txt  
  inflating: stanford-ner-2020-11-17/build.xml  
  inflating: stanford-ner-2020-11-17/ner-gui.bat  
  inflating: stanford-ner-2020-11-17/sample-w-time.txt  
  inflating: stanford-ner-2020-11-17/ner-gui.command  
   cr

# Configurations

In [ ]:
class configs:
  data_path = '/content/news_sample_ner.txt'
  cleaned_data_path = '/content/news_sample_ner_cleaned.txt'
  stanford_ner_model_path = '/content/stanford-ner-2020-11-17/classifiers/english.all.3class.distsim.crf.ser.gz'
  stanford_ner_jar_path = '/content/stanford-ner-2020-11-17/stanford-ner.jar'


# Imports

In [ ]:
from bs4 import BeautifulSoup
import nltk
from nltk.tag import StanfordNERTagger
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
import spacy
from spacy import displacy
from spacy import tokenizer
from tqdm import tqdm
from itertools import groupby

# Data Cleaning and Preparation

In [ ]:
def read_file(path):
    """Reads a text file from the disk.

    Args:
      path: Path to the file

    Returns:
      data: Text of the file
    """
    text_file = open(path, "r")
    data = text_file.read()
    text_file.close()
    return data

def write_file(path, data):
    """Writes a file to the disk.

      Args:
        path: path of writing the file
        data: Text to be written
    """
    text_file = open(path, "w")
    n = text_file.write(data)
    text_file.close()

def clean_data(data):
    """Cleans text from HTML tags.

    Args:
      data: Text to be cleaned

    Returns:
      Cleaned text
    """
    soup = BeautifulSoup(data, "html.parser")
    for txt in soup(['style', 'script']):
        txt.decompose()
    return ' '.join(soup.stripped_strings)

def get_sentences(data):
  """Tokenizes the data into sentences.

    Args:
      data: Text to be tokenized

    Returns:
      sentences: Tokenized sentences
  """
  htmlParse = BeautifulSoup(data, 'html.parser')
  sentences = []
    
  for para in htmlParse.find_all("p"):
      para_txt = para.get_text()
      sentences_para = nltk.sent_tokenize(para_txt)
      for sent in sentences_para:
        sentences.append(sent.replace('\n', ' '))

  return sentences

def get_labels(data):
  """Gets the named entities from the data.

    Args:
      data: Text to get the named entities from

    Returns:
      labels: A dictionary where keys are: locations, organizations and persons, and values are the entities corresponding to each key
  """
  htmlParse = BeautifulSoup(data, 'html.parser')
  labels = {'locations':[], 'organizations':[], 'persons':[]}

  for i in htmlParse.findAll(attrs={'type' : 'LOCATION'}):
      labels['locations'].append(i.text.replace('\n', ' '))

  for i in htmlParse.findAll(attrs={'type' : 'ORGANIZATION'}):
      labels['organizations'].append(i.text.replace('\n', ' '))

  for i in htmlParse.findAll(attrs={'type' : 'PERSON'}):
      labels['persons'].append(i.text.replace('\n', ' '))
  print("Entities distribution:")

  print("Locations: {}\nOrganizations: {}\nPersons: {}".format(len(labels['locations']), len(labels['organizations']), len(labels['persons'])))

  print("\nUnique words entities distribution:")

  print("Locations: {}\nOrganizations: {}\nPersons: {}".format(len(set(labels['locations'])), len(set(labels['organizations'])), len(set(labels['persons']))))

  return labels

In [ ]:
data = read_file(configs.data_path)

In [ ]:
cleaned_data = clean_data(data)
write_file(configs.cleaned_data_path, cleaned_data)

In [ ]:
sentences = get_sentences(data)

In [ ]:
labels = get_labels(data)

Entities distribution:
Locations: 167
Organizations: 173
Persons: 93

Unique words entities distribution:
Locations: 81
Organizations: 70
Persons: 64


# Evaluation Metrics

In [ ]:
def calc_precision(preds, labels, entity):
  """Calculates precision.

    Args:
      preds: A prediction dictionary where keys are locations, organizations and persons, and values are the predicted entities for each key
      labels: A ground truth dictionary where keys are locations, organizations and persons, and values are the ground truth entities for each key
      entity: The entity to calculate precision for (locations or organizations or persons)

    Returns:
      The calculated precision for the entity
  """
  cnt = 0
  for pred in preds[entity]:
    if pred in labels[entity]:
      cnt+=1
  return cnt / len(preds[entity])

In [ ]:
def calc_recall(preds, labels, entity):
  """Calculates recall.

    Args:
      preds: A prediction dictionary where keys are locations, organizations and persons, and values are the predicted entities for each key
      labels: A ground truth dictionary where keys are locations, organizations and persons, and values are the ground truth entities for each key
      entity: The entity to calculate recall for (locations or organizations or persons)

    Returns:
      The calculated recall for the entity
  """
  cnt = 0
  for label in labels[entity]:
    if label in preds[entity]:
      cnt+=1
  return cnt / len(labels[entity])

In [ ]:
def calc_f1_score(precision, recall):
  """Calculates F1-score.

    Args:
      precision: The calculated precision
      recall: The calculated recall

    Returns:
      The calculated F1-score
  """
  return (2 * precision * recall) / (precision + recall)

In [ ]:
def calc_metrics(preds, labels):
  """Reports precision, recall and F1-score.

    Args:
      preds: A prediction dictionary where keys are locations, organizations and persons, and values are the predicted entities for each key
      labels: A ground truth dictionary where keys are locations, organizations and persons, and values are the ground truth entities for each key
  """

  person_precision = calc_precision(preds, labels, 'persons')
  locations_precision = calc_precision(preds, labels, 'locations')
  organizations_precision = calc_precision(preds, labels, 'organizations')

  person_recall = calc_recall(preds, labels, 'persons')
  locations_recall = calc_recall(preds, labels, 'locations')
  organizations_recall = calc_recall(preds, labels, 'organizations')

  person_f1_score = calc_f1_score(person_precision, person_recall)
  locations_f1_score = calc_f1_score(locations_precision, locations_recall)
  organizations_f1_score = calc_f1_score(organizations_precision, organizations_recall)

  print('***** Persons *****')
  print('Precision: {:.3f} \t Recall: {:.3f} \t F1-score: {:.3f}'.format(person_precision, person_recall, person_f1_score))
  print('\n')
  print('***** Locations *****')
  print('Precision: {:.3f} \t Recall: {:.3f} \t F1-score: {:.3f}'.format(locations_precision, locations_recall, locations_f1_score))
  print('\n')
  print('***** Organizations *****')
  print('Precision: {:.3f} \t Recall: {:.3f} \t F1-score: {:.3f}'.format(organizations_precision, organizations_recall, organizations_f1_score))

# Prediction Functions

In [ ]:
def predict_nltk_stanford_ner(sentences, model_path, jar_path):
  """Predicts named entities from a list of sentences based on Stanford NER model.

    Args:
      sentences: A list of tokenized sentences
      model_path: Path to model
      jar_path: Path to jar file

    Returns:
      preds_ntlk: A prediction dictionary where keys are locations, organizations and persons, and values are the predicted entities for each key
  """
  model = model_path
  jar = jar_path
  #model = '/content/stanford-ner-2020-11-17/classifiers/english.all.3class.distsim.crf.ser.gz'
  #jar = '/content/stanford-ner-2020-11-17/stanford-ner.jar'
  st = StanfordNERTagger(model, jar,encoding='utf-8')

  preds_ntlk = {'locations':[], 'organizations':[], 'persons':[]}

  for sentence in tqdm(sentences):
    tokenized_text = nltk.word_tokenize(sentence)
    classified_text = st.tag(tokenized_text)

    entities = []
    labels = []
    
    for tag, chunk in groupby(classified_text, lambda x:x[1]):
        if tag != "O":
            entities.append(' '.join(w for w, t in chunk))
            labels.append(tag)
    

    for entity, label in zip(entities, labels):
      if label == 'LOCATION':
        preds_ntlk['locations'].append(entity)
      elif label == 'ORGANIZATION':
        preds_ntlk['organizations'].append(entity)
      elif label == 'PERSON':
        preds_ntlk['persons'].append(entity)
  
  print("\nEntities distribution:")

  print("Locations: {}\nOrganizations: {}\nPersons: {}".format(len(preds_ntlk['locations']), len(preds_ntlk['organizations']), len(preds_ntlk['persons'])))

  print("\nUnique words entities distribution:")

  print("Locations: {}\nOrganizations: {}\nPersons: {}".format(len(set(preds_ntlk['locations'])), len(set(preds_ntlk['organizations'])), len(set(preds_ntlk['persons']))))
  
  return preds_ntlk

In [ ]:
def predict_nltk(sentences):
  """Predicts named entities from a list of sentences based on NLTK NER model.

    Args:
      sentences: A list of tokenized sentences

    Returns:
      preds_ntlk: A prediction dictionary where keys are locations, organizations and persons, and values are the predicted entities for each key
  """
  preds_nltk = {'locations':[], 'organizations':[], 'persons':[]}

  for sentence in tqdm(sentences):
    words = nltk.word_tokenize(sentence)
    tagged = nltk.pos_tag(words)
    chunks = nltk.ne_chunk(tagged, binary=False)

    for chunk in chunks:
      if hasattr(chunk,'label'):
          if chunk.label() == 'LOCATION' or chunk.label() == 'GPE':
            preds_nltk['locations'].append(' '.join(c[0] for c in chunk))
          elif chunk.label() == 'ORGANIZATION':
            preds_nltk['organizations'].append(' '.join(c[0] for c in chunk))
          elif chunk.label() == 'PERSON':
            preds_nltk['persons'].append(' '.join(c[0] for c in chunk))
  
  print("\nEntities distribution:")

  print("Locations: {}\nOrganizations: {}\nPersons: {}".format(len(preds_nltk['locations']), len(preds_nltk['organizations']), len(preds_nltk['persons'])))

  print("\nUnique words entities distribution:")

  print("Locations: {}\nOrganizations: {}\nPersons: {}".format(len(set(preds_nltk['locations'])), len(set(preds_nltk['organizations'])), len(set(preds_nltk['persons']))))
  
  return preds_nltk

In [ ]:
def predict_spacy(sentences, transformer_based = 0):
  """Predicts named entities from a list of sentences based on Spacy NER model.

    Args:
      sentences: A list of tokenized sentences
      transformer_based: A flag to choose whether to use the transformer basde model or not
      1: Use en_core_web_trf model
      0: Use en_core_web_sm model

    Returns:
      preds_spacy: A prediction dictionary where keys are locations, organizations and persons, and values are the predicted entities for each key
  """
  if transformer_based:
    nlp = spacy.load('en_core_web_trf')
  else:
    nlp = spacy.load('en_core_web_sm')
    
  preds_spacy = {'locations':[], 'organizations':[], 'persons':[]}

  for sentence in tqdm(sentences):
    doc = nlp(sentence)
    ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
    for ent in ents:
      if ent[-1] == 'LOC' or ent[-1] == 'GPE':
        preds_spacy['locations'].append(ent[0])
      elif ent[-1] == 'ORG':
        preds_spacy['organizations'].append(ent[0])
      elif ent[-1] == 'PERSON':
        preds_spacy['persons'].append(ent[0])
  
  print("\nEntities distribution:")

  print("Locations: {}\nOrganizations: {}\nPersons: {}".format(len(preds_spacy['locations']), len(preds_spacy['organizations']), len(preds_spacy['persons'])))

  print("\nUnique words entities distribution:")

  print("Locations: {}\nOrganizations: {}\nPersons: {}".format(len(set(preds_spacy['locations'])), len(set(preds_spacy['organizations'])), len(set(preds_spacy['persons']))))
  
  return preds_spacy

# NER with Statistical Models

Stanford NER Model

In [ ]:
preds_nltk_stanford_ner = predict_nltk_stanford_ner(sentences, configs.stanford_ner_model_path, configs.stanford_ner_jar_path)

100%|██████████| 2578/2578 [2:13:45<00:00,  3.11s/it]


Entities distribution:
Locations: 1233
Organizations: 1231
Persons: 795

Unique words entities distribution:
Locations: 73
Organizations: 60
Persons: 57


In [ ]:
calc_metrics(preds_nltk_stanford_ner, labels)

***** Persons *****
Precision: 0.964 	 Recall: 0.839 	 F1-score: 0.897


***** Locations *****
Precision: 0.972 	 Recall: 0.898 	 F1-score: 0.934


***** Organizations *****
Precision: 0.944 	 Recall: 0.792 	 F1-score: 0.861


Spacy NER Model

In [ ]:
preds_spacy = predict_spacy(sentences, transformer_based=0)

100%|██████████| 2578/2578 [00:26<00:00, 98.29it/s]


Entities distribution:
Locations: 1181
Organizations: 1466
Persons: 580

Unique words entities distribution:
Locations: 69
Organizations: 91
Persons: 52


In [ ]:
calc_metrics(preds_spacy, labels)

***** Persons *****
Precision: 0.772 	 Recall: 0.570 	 F1-score: 0.656


***** Locations *****
Precision: 0.820 	 Recall: 0.802 	 F1-score: 0.811


***** Organizations *****
Precision: 0.538 	 Recall: 0.676 	 F1-score: 0.599


NLTK NER Model

In [ ]:
preds_nltk = predict_nltk(sentences)

100%|██████████| 2578/2578 [00:27<00:00, 94.02it/s]


Entities distribution:
Locations: 1726
Organizations: 1079
Persons: 1015

Unique words entities distribution:
Locations: 93
Organizations: 73
Persons: 79


In [ ]:
calc_metrics(preds_nltk, labels)

***** Persons *****
Precision: 0.622 	 Recall: 0.710 	 F1-score: 0.663


***** Locations *****
Precision: 0.492 	 Recall: 0.784 	 F1-score: 0.605


***** Organizations *****
Precision: 0.466 	 Recall: 0.561 	 F1-score: 0.509


# NER with Transformers

Spacy Transformer Based NER Model

In [ ]:
preds_spacy_trf = predict_spacy(sentences, transformer_based=1)

  0%|          | 0/2578 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
100%|██████████| 2578/2578 [07:06<00:00,  6.04it/s]


Entities distribution:
Locations: 1264
Organizations: 1369
Persons: 737

Unique words entities distribution:
Locations: 74
Organizations: 80
Persons: 57


In [ ]:
calc_metrics(preds_spacy_trf, labels)

***** Persons *****
Precision: 0.984 	 Recall: 0.849 	 F1-score: 0.912


***** Locations *****
Precision: 0.828 	 Recall: 0.868 	 F1-score: 0.848


***** Organizations *****
Precision: 0.684 	 Recall: 0.763 	 F1-score: 0.722
